# 当数据很多时，内存资源显得十分宝贵，合理利用内存非常重要

In [1]:
import datetime
import os
import threading
from pymongo import MongoClient

In [2]:
threadNum = 128
folderNum = 8
osm = MongoClient('192.168.168.7', 5077)
au = osm.admin.authenticate("sa","6hNmEVCd",mechanism="SCRAM-SHA-1")
db = osm.aisdata
col = db.shipInfo
erCol = db.shipInfoError

In [3]:
DATA = "/root/ais/data/2018/trajectory%s/" %( folderNum )  #数据文件
TASK = "/root/ais/Projects/month%s/" %( folderNum )      #工作路径

#  1.分割任务
 **将每月的数据分为foldernum份，每一份又分为threadNum份**

In [4]:
tasksplit = 10        #分割为10个文件夹
createVar = locals()
dataDir = DATA
taskListDir = TASK
taskdir = os.path.join(taskListDir, "folder%02d" % (folderNum)) 
Var = locals()
filePath = []

## 扫描所有数据文件

In [5]:
if os.path.exists(taskdir):
    print("spliting files finashed.\n")
else:
    os.mkdir(taskdir)
for root, dirs, files in os.walk(dataDir):      #扫描所有数据文件名
    for file in files:  
        if file.endswith('txt'):
            filePath.append(os.path.join(root, file))
print("All txt: %d" % ( len(filePath) ))

spliting files finashed.

All txt: 84990


## 将所有将要插入的数据分为tasksplit个

In [6]:
for i in range(tasksplit):          #生成tasksplit个list 
    queue = "%02d" % ( i )
    createVar["fileQueue" + queue] = []    

In [7]:
for i in range(len(filePath)):      #分配task给每一个tasksplit
    queue = "%02d" % (i % tasksplit)
    createVar["fileQueue" + queue].append(filePath[i])

In [8]:
for i in range(tasksplit):          #打印出每个tasksplit长度
    queue = "%02d" % ( i )
    print(len(createVar["fileQueue" + queue]))

8499
8499
8499
8499
8499
8499
8499
8499
8499
8499


## 将每个tasksplit分为threadNum份

In [9]:
for i in range(tasksplit):          #生成文件夹
    queue = "%02d" % ( i )
    taskdirnum = os.path.join(taskdir,"Task%s"%(queue))
    if not os.path.isdir(taskdirnum):
        os.mkdir(taskdirnum)

In [10]:
for m in range(threadNum*tasksplit):          #256*10个Var[]
    Vqu = "%02d" % ( m )
    Var["fqueue" + Vqu] = []
for j in range(tasksplit):      #0
    ques = "%02d" % ( j )         #0
    cVarlen = len(createVar["fileQueue" + ques])   #16330
    queVar = j*threadNum   # 256*j 
    for k in range(cVarlen):      #16330
        Vqu = k % threadNum
        Varque = "%02d" % (Vqu+queVar)    #64
        Var["fqueue" + Varque].append(createVar["fileQueue" + ques][k])    #64*256

In [11]:
len(Var["fqueue25"])

KeyError: 'fqueue2559'

## 分配所有任务到每一个文件夹里

In [13]:
for i in range(tasksplit):          
    queue = "%02d" % ( i )    #0
    queVar = i*threadNum
    #cVarlen = len(createVar["fileQueue" + queue])   #
    taskfolder = os.path.join(taskdir ,"Task%s"%(queue))   #Task00
    for j in range(threadNum):      #0
        que = "%02d" % ( j )         #0
        
        fn = os.path.join(taskfolder,"task%s.txt" % (que))   #task00
        if os.path.isfile(fn):                              
            os.remove(fn)
            
        Varque = "%02d" %(queVar+j)
        with open(fn, "a") as task:                   #task00
            for file in Var["fqueue" + Varque]:      #Var[00]
                task.write(file + "\n")
                
        print("Task%s folder task%s.txt has %d pieces task." % (queue, que, len(Var["fqueue" + que])))   #Task00 folder task00.txt has 6 pieces task.
        Var["fileQueue" + Varque] = []    #Var[00] =[]
        fn = []       #task00 = []

    createVar["fileQueue" + queue] = []    #createVar[00]
    print("Task%s folder spliting files finished.\n"%(queue))
    
print("all folder spliting files finished.\n")
filePath = []

Task00 folder task00.txt has 66 pieces task.
Task00 folder task01.txt has 66 pieces task.
Task00 folder task02.txt has 66 pieces task.
Task00 folder task03.txt has 66 pieces task.
Task00 folder task04.txt has 66 pieces task.
Task00 folder task05.txt has 66 pieces task.
Task00 folder task06.txt has 66 pieces task.
Task00 folder task07.txt has 66 pieces task.
Task00 folder task08.txt has 66 pieces task.
Task00 folder task09.txt has 66 pieces task.
Task00 folder task10.txt has 66 pieces task.
Task00 folder task11.txt has 66 pieces task.
Task00 folder task12.txt has 66 pieces task.
Task00 folder task13.txt has 66 pieces task.
Task00 folder task14.txt has 66 pieces task.
Task00 folder task15.txt has 66 pieces task.
Task00 folder task16.txt has 66 pieces task.
Task00 folder task17.txt has 66 pieces task.
Task00 folder task18.txt has 66 pieces task.
Task00 folder task19.txt has 66 pieces task.
Task00 folder task20.txt has 66 pieces task.
Task00 folder task21.txt has 66 pieces task.
Task00 fol

Task03 folder task125.txt has 65 pieces task.
Task03 folder task126.txt has 65 pieces task.
Task03 folder task127.txt has 65 pieces task.
Task03 folder spliting files finished.

Task04 folder task00.txt has 66 pieces task.
Task04 folder task01.txt has 66 pieces task.
Task04 folder task02.txt has 66 pieces task.
Task04 folder task03.txt has 66 pieces task.
Task04 folder task04.txt has 66 pieces task.
Task04 folder task05.txt has 66 pieces task.
Task04 folder task06.txt has 66 pieces task.
Task04 folder task07.txt has 66 pieces task.
Task04 folder task08.txt has 66 pieces task.
Task04 folder task09.txt has 66 pieces task.
Task04 folder task10.txt has 66 pieces task.
Task04 folder task11.txt has 66 pieces task.
Task04 folder task12.txt has 66 pieces task.
Task04 folder task13.txt has 66 pieces task.
Task04 folder task14.txt has 66 pieces task.
Task04 folder task15.txt has 66 pieces task.
Task04 folder task16.txt has 66 pieces task.
Task04 folder task17.txt has 66 pieces task.
Task04 folde

Task08 folder task63.txt has 66 pieces task.
Task08 folder task64.txt has 66 pieces task.
Task08 folder task65.txt has 66 pieces task.
Task08 folder task66.txt has 66 pieces task.
Task08 folder task67.txt has 66 pieces task.
Task08 folder task68.txt has 66 pieces task.
Task08 folder task69.txt has 66 pieces task.
Task08 folder task70.txt has 66 pieces task.
Task08 folder task71.txt has 66 pieces task.
Task08 folder task72.txt has 66 pieces task.
Task08 folder task73.txt has 66 pieces task.
Task08 folder task74.txt has 66 pieces task.
Task08 folder task75.txt has 66 pieces task.
Task08 folder task76.txt has 66 pieces task.
Task08 folder task77.txt has 66 pieces task.
Task08 folder task78.txt has 66 pieces task.
Task08 folder task79.txt has 66 pieces task.
Task08 folder task80.txt has 66 pieces task.
Task08 folder task81.txt has 66 pieces task.
Task08 folder task82.txt has 66 pieces task.
Task08 folder task83.txt has 66 pieces task.
Task08 folder task84.txt has 66 pieces task.
Task08 fol

Task13 folder task29.txt has 66 pieces task.
Task13 folder task30.txt has 66 pieces task.
Task13 folder task31.txt has 66 pieces task.
Task13 folder task32.txt has 66 pieces task.
Task13 folder task33.txt has 66 pieces task.
Task13 folder task34.txt has 66 pieces task.
Task13 folder task35.txt has 66 pieces task.
Task13 folder task36.txt has 66 pieces task.
Task13 folder task37.txt has 66 pieces task.
Task13 folder task38.txt has 66 pieces task.
Task13 folder task39.txt has 66 pieces task.
Task13 folder task40.txt has 66 pieces task.
Task13 folder task41.txt has 66 pieces task.
Task13 folder task42.txt has 66 pieces task.
Task13 folder task43.txt has 66 pieces task.
Task13 folder task44.txt has 66 pieces task.
Task13 folder task45.txt has 66 pieces task.
Task13 folder task46.txt has 66 pieces task.
Task13 folder task47.txt has 66 pieces task.
Task13 folder task48.txt has 66 pieces task.
Task13 folder task49.txt has 66 pieces task.
Task13 folder task50.txt has 66 pieces task.
Task13 fol

Task18 folder task00.txt has 66 pieces task.
Task18 folder task01.txt has 66 pieces task.
Task18 folder task02.txt has 66 pieces task.
Task18 folder task03.txt has 66 pieces task.
Task18 folder task04.txt has 66 pieces task.
Task18 folder task05.txt has 66 pieces task.
Task18 folder task06.txt has 66 pieces task.
Task18 folder task07.txt has 66 pieces task.
Task18 folder task08.txt has 66 pieces task.
Task18 folder task09.txt has 66 pieces task.
Task18 folder task10.txt has 66 pieces task.
Task18 folder task11.txt has 66 pieces task.
Task18 folder task12.txt has 66 pieces task.
Task18 folder task13.txt has 66 pieces task.
Task18 folder task14.txt has 66 pieces task.
Task18 folder task15.txt has 66 pieces task.
Task18 folder task16.txt has 66 pieces task.
Task18 folder task17.txt has 66 pieces task.
Task18 folder task18.txt has 66 pieces task.
Task18 folder task19.txt has 66 pieces task.
Task18 folder task20.txt has 66 pieces task.
Task18 folder task21.txt has 66 pieces task.
Task18 fol

# 2.数据插入

In [17]:
def insertData(data, col, erCol,col_insert, erCol_insert):      #数据插入
    if not data == "":
        try:
            q = data.split(';')
            dic = {
                "id": int(q[0]),
                "date": str(q[1].split(" ")[0]),
                "time": str(q[1].split(" ")[1]),
                "lon": float(q[2]),
                "lat": float(q[3]),
                "way": float(q[4]),
                "speed": float(q[5]),
                "sid": int(q[6])
            }
            #print(dic)
            col_insert.append(dic)

            #col.insert_one(dic)

            return 0

        except:
            er = {
                "data": str(data.strip('\n')),
                "length": int(len(q))
            }
            erCol_insert.append(er)
            #erCol.insert_one(er)

            return 1

    else:
        if len(col_insert):
            col.insert_many(col_insert)
        if len(erCol_insert):
            erCol.insert_many(erCol_insert)

        return 0


In [18]:
class threadInsert(threading.Thread):                  #多线程插入
    def __init__(self, queue, fileList, col, erCol):
        threading.Thread.__init__(self)
        self.queue = queue
        self.fileList = fileList
        self.col = col
        self.erCol = erCol

    def run(self):
        error = []
        total = 0
        with open(self.fileList, "r") as taskList:
            taskList = taskList.read().split("\n")
            
        for txt in taskList:
            if txt == "":
                continue
            else:
                with open(txt, "r") as data:
                    data = data.read().split("\n")
                month = os.path.dirname(txt)[-11:]
                print("%s thread %s assigned %s pieces of data." % (month, self.queue, len(data) ))
                total += len(data)

                col_insert = []
                erCol_insert = []
                for c in data:
                    a = insertData(c, self.col, self.erCol, col_insert, erCol_insert)
                    
        self.result = total

    def getResult(self):
        try:
            return self.result
        except:
            return Exception

In [19]:
 def main(foldernum):
    START = datetime.datetime.now()
    strSTART = START.strftime("%Y-%m-%d %H:%M:%S")
    with open (os.path.join(TASK, "%02d_time.log" % ( folderNum )), "a") as l:
        l.write(strSTART + "\n")
        
    TOTAL = 0
    threads = []

    try:
        for i in range(threadNum):
            queue = "%02d" % ( i )
            taskFile = os.path.join(TASK, "folder%02d" %(folderNum), "Task%02d"%(foldernum),"task%s.txt" % ( queue ))
            strtask = "Task%02d "%(foldernum) + taskFile[-10:-4]
            print(strtask)
            t = threadInsert(queue=i, fileList=taskFile, col=col, erCol=erCol)
            t.start()
            threads.append(t)
        for t in threads:
            t.join()
            result = t.getResult()
            TOTAL += result
            print("finished %d pieces of data." % ( result ))
            # print("%d pieces of data finished, %d errors." % ( col.find().count(), erCol.find().count() ))

    except:
        CUR = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with open(os.path.join(TASK, "error.log"), "a") as erLog:
            erLog.write(CUR + "\n")
            erLog.write("%02d" % ( folderNum ) + " - " + str(Exception) + "\n")
    END = datetime.datetime.now()
    strEND = END.strftime("%Y-%m-%d %H:%M:%S")
    with open (os.path.join(TASK, "%02d_time.log" % ( folderNum )), "a") as tlog:
        tlog.write(strEND)
    
    DELTA = (END - START).seconds
    print("All threads finished, used %s seconds." % (DELTA))
    print("%d pieces of data (%d) has been inserted, %d errors happened." %( Total, col.find().count(), erCol.find().count() ))


# 经过多次尝试，用了一个虽然很笨，但是最有效的方法，重复十次main(*)

In [ ]:
main(0)
main(1)
main(2)
main(3)
main(4)
main(5)
main(6)
main(7)
main(8)
main(9)

Task00 task00
Task00 task01
Task00 task02
Task00 task03
Task00 task04
Task00 task05
trajectory6 thread 0 assigned 3 pieces of data.
trajectory6 thread 1 assigned 7 pieces of data.
Task00 task06trajectory6 thread 4 assigned 2 pieces of data.

trajectory6 thread 2 assigned 2 pieces of data.
Task00 task07
trajectory6 thread 5 assigned 5 pieces of data.
trajectory6 thread 3 assigned 2 pieces of data.
trajectory6 thread 0 assigned 2 pieces of data.
trajectory6 thread 4 assigned 681 pieces of data.
Task00 task08
trajectory6 thread 6 assigned 46 pieces of data.
Task00 task09
Task00 task10
Task00 task11
trajectory6 thread 7 assigned 2 pieces of data.
Task00 task12
Task00 task13
Task00 task14
Task00 task15trajectory6 thread 8 assigned 88 pieces of data.
trajectory6 thread 10 assigned 8 pieces of data.
trajectory6 thread 9 assigned 3 pieces of data.

Task00 task16
trajectory6 thread 11 assigned 2 pieces of data.
Task00 task17trajectory6 thread 12 assigned 2 pieces of data.

trajectory6 thread 2 

trajectory6 thread 117 assigned 497 pieces of data.
Task00 ask124trajectory6 thread 12 assigned 2 pieces of data.
trajectory6 thread 121 assigned 3393 pieces of data.
trajectory6 thread 120 assigned 2 pieces of data.
trajectory6 thread 119 assigned 2026 pieces of data.

trajectory6 thread 122 assigned 4 pieces of data.
Task00 ask125
trajectory6 thread 123 assigned 2 pieces of data.
Task00 ask126
trajectory6 thread 22 assigned 13384 pieces of data.
trajectory6 thread 125 assigned 454 pieces of data.
Task00 ask127
trajectory6 thread 126 assigned 301 pieces of data.
trajectory6 thread 124 assigned 2 pieces of data.
trajectory6 thread 127 assigned 21634 pieces of data.
trajectory6 thread 38 assigned 8132 pieces of data.
trajectory6 thread 1 assigned 2 pieces of data.
trajectory6 thread 23 assigned 12422 pieces of data.
trajectory6 thread 17 assigned 2 pieces of data.
trajectory6 thread 19 assigned 2 pieces of data.
trajectory6 thread 24 assigned 3 pieces of data.
trajectory6 thread 34 assi

trajectory6 thread 52 assigned 37146 pieces of data.trajectory6 thread 30 assigned 195 pieces of data.

trajectory6 thread 36 assigned 3 pieces of data.
trajectory6 thread 85 assigned 2 pieces of data.trajectory6 thread 27 assigned 3 pieces of data.

trajectory6 thread 72 assigned 124524 pieces of data.
trajectory6 thread 115 assigned 202379 pieces of data.
trajectory6 thread 105 assigned 1239 pieces of data.
trajectory6 thread 80 assigned 6 pieces of data.
trajectory6 thread 85 assigned 2 pieces of data.
trajectory6 thread 18 assigned 149895 pieces of data.
trajectory6 thread 49 assigned 2 pieces of data.
trajectory6 thread 41 assigned 4997 pieces of data.
trajectory6 thread 27 assigned 2 pieces of data.
trajectory6 thread 85 assigned 12505 pieces of data.trajectory6 thread 36 assigned 49382 pieces of data.

trajectory6 thread 49 assigned 154317 pieces of data.
trajectory6 thread 27 assigned 385 pieces of data.
trajectory6 thread 124 assigned 24241 pieces of data.
trajectory6 thread 8

trajectory6 thread 15 assigned 15 pieces of data.
trajectory6 thread 15 assigned 3 pieces of data.
trajectory6 thread 15 assigned 31335 pieces of data.
trajectory6 thread 21 assigned 102424 pieces of data.
trajectory6 thread 106 assigned 1690 pieces of data.
trajectory6 thread 67 assigned 7126 pieces of data.
trajectory6 thread 70 assigned 507 pieces of data.
trajectory6 thread 24 assigned 2 pieces of data.
trajectory6 thread 24 assigned 300 pieces of data.
trajectory6 thread 109 assigned 4 pieces of data.
trajectory6 thread 109 assigned 38825 pieces of data.
trajectory6 thread 94 assigned 837 pieces of data.
trajectory6 thread 114 assigned 6273 pieces of data.
trajectory6 thread 73 assigned 1258 pieces of data.
trajectory6 thread 119 assigned 944 pieces of data.
trajectory6 thread 24 assigned 2 pieces of data.
trajectory6 thread 24 assigned 15549 pieces of data.
trajectory6 thread 110 assigned 40529 pieces of data.
trajectory6 thread 70 assigned 10081 pieces of data.
trajectory6 threa

trajectory6 thread 12 assigned 6 pieces of data.
trajectory6 thread 12 assigned 3281 pieces of data.
trajectory6 thread 92 assigned 120 pieces of data.
trajectory6 thread 92 assigned 5001 pieces of data.
trajectory6 thread 105 assigned 3 pieces of data.
trajectory6 thread 105 assigned 2 pieces of data.
trajectory6 thread 105 assigned 18848 pieces of data.
trajectory6 thread 37 assigned 42915 pieces of data.
trajectory6 thread 106 assigned 4 pieces of data.
trajectory6 thread 6 assigned 2 pieces of data.
trajectory6 thread 6 assigned 3 pieces of data.
trajectory6 thread 106 assigned 2 pieces of data.
trajectory6 thread 6 assigned 3 pieces of data.
trajectory6 thread 106 assigned 2 pieces of data.
trajectory6 thread 6 assigned 2 pieces of data.
trajectory6 thread 106 assigned 14 pieces of data.
trajectory6 thread 6 assigned 86069 pieces of data.
trajectory6 thread 106 assigned 6 pieces of data.
trajectory6 thread 106 assigned 1638 pieces of data.
trajectory6 thread 67 assigned 31 pieces 

...

trajectory6 thread 10 assigned 2 pieces of data.
trajectory6 thread 10 assigned 2 pieces of data.
trajectory6 thread 10 assigned 2 pieces of data.
trajectory6 thread 10 assigned 9 pieces of data.
trajectory6 thread 89 assigned 15 pieces of data.
trajectory6 thread 10 assigned 2 pieces of data.
trajectory6 thread 10 assigned 2 pieces of data.
trajectory6 thread 10 assigned 28345 pieces of data.
trajectory6 thread 89 assigned 2 pieces of data.
trajectory6 thread 89 assigned 2438 pieces of data.
trajectory6 thread 8 assigned 3 pieces of data.
trajectory6 thread 8 assigned 2 pieces of data.
trajectory6 thread 8 assigned 1844 pieces of data.
trajectory6 thread 64 assigned 3 pieces of data.
trajectory6 thread 64 assigned 3604 pieces of data.
trajectory6 thread 6 assigned 2 pieces of data.
trajectory6 thread 6 assigned 14722 pieces of data.
trajectory6 thread 19 assigned 2644 pieces of data.
trajectory6 thread 41 assigned 3180 pieces of data.
trajectory6 thread 116 assigned 2145 pieces of dat

trajectory6 thread 47 assigned 3367 pieces of data.
trajectory6 thread 54 assigned 2 pieces of data.
trajectory6 thread 54 assigned 2 pieces of data.
trajectory6 thread 54 assigned 20 pieces of data.
trajectory6 thread 54 assigned 2 pieces of data.
trajectory6 thread 54 assigned 2 pieces of data.
trajectory6 thread 54 assigned 2 pieces of data.
trajectory6 thread 54 assigned 2 pieces of data.
trajectory6 thread 54 assigned 2 pieces of data.
trajectory6 thread 54 assigned 2 pieces of data.
trajectory6 thread 54 assigned 2841 pieces of data.
trajectory6 thread 27 assigned 219931 pieces of data.
trajectory6 thread 120 assigned 37559 pieces of data.
trajectory6 thread 88 assigned 919 pieces of data.
trajectory6 thread 14 assigned 3 pieces of data.
trajectory6 thread 14 assigned 4265 pieces of data.
trajectory6 thread 111 assigned 2787 pieces of data.
trajectory6 thread 92 assigned 5 pieces of data.
trajectory6 thread 92 assigned 1180 pieces of data.
trajectory6 thread 125 assigned 8 pieces

trajectory6 thread 17 assigned 3 pieces of data.
trajectory6 thread 19 assigned 2 pieces of data.
trajectory6 thread 19 assigned 6 pieces of data.trajectory6 thread 17 assigned 26 pieces of data.

trajectory6 thread 19 assigned 23920 pieces of data.
trajectory6 thread 17 assigned 2 pieces of data.
trajectory6 thread 17 assigned 9 pieces of data.
trajectory6 thread 17 assigned 8 pieces of data.
trajectory6 thread 17 assigned 2 pieces of data.
trajectory6 thread 17 assigned 2 pieces of data.
trajectory6 thread 17 assigned 21 pieces of data.
trajectory6 thread 17 assigned 4 pieces of data.
trajectory6 thread 54 assigned 4689 pieces of data.
trajectory6 thread 17 assigned 2 pieces of data.
trajectory6 thread 17 assigned 2 pieces of data.
trajectory6 thread 17 assigned 2 pieces of data.
trajectory6 thread 17 assigned 2 pieces of data.
trajectory6 thread 17 assigned 12956 pieces of data.
trajectory6 thread 66 assigned 2558 pieces of data.
trajectory6 thread 7 assigned 3 pieces of data.
traje

trajectory6 thread 16 assigned 8382 pieces of data.
trajectory6 thread 39 assigned 2 pieces of data.
trajectory6 thread 39 assigned 4 pieces of data.
trajectory6 thread 39 assigned 4147 pieces of data.
trajectory6 thread 109 assigned 4414 pieces of data.
trajectory6 thread 74 assigned 4725 pieces of data.
trajectory6 thread 118 assigned 5 pieces of data.
trajectory6 thread 118 assigned 2 pieces of data.
trajectory6 thread 118 assigned 20 pieces of data.
trajectory6 thread 118 assigned 2 pieces of data.
trajectory6 thread 118 assigned 4 pieces of data.
trajectory6 thread 118 assigned 2 pieces of data.
trajectory6 thread 118 assigned 118 pieces of data.
trajectory6 thread 121 assigned 75 pieces of data.
trajectory6 thread 121 assigned 4024 pieces of data.
trajectory6 thread 118 assigned 5407 pieces of data.
trajectory6 thread 87 assigned 6051 pieces of data.
trajectory6 thread 54 assigned 60 pieces of data.
trajectory6 thread 11 assigned 7 pieces of data.
trajectory6 thread 11 assigned 2

trajectory6 thread 116 assigned 2 pieces of data.
trajectory6 thread 116 assigned 2 pieces of data.
trajectory6 thread 116 assigned 4 pieces of data.
trajectory6 thread 116 assigned 14043 pieces of data.
trajectory6 thread 87 assigned 3194 pieces of data.
trajectory6 thread 26 assigned 41849 pieces of data.
trajectory6 thread 14 assigned 147102 pieces of data.
trajectory6 thread 91 assigned 2613 pieces of data.
trajectory6 thread 11 assigned 396 pieces of data.
trajectory6 thread 54 assigned 14 pieces of data.
trajectory6 thread 54 assigned 10 pieces of data.
trajectory6 thread 54 assigned 2007 pieces of data.
trajectory6 thread 68 assigned 3 pieces of data.
trajectory6 thread 68 assigned 2 pieces of data.
trajectory6 thread 68 assigned 27233 pieces of data.
trajectory6 thread 16 assigned 199 pieces of data.
trajectory6 thread 17 assigned 6 pieces of data.
trajectory6 thread 17 assigned 15905 pieces of data.
trajectory6 thread 11 assigned 3 pieces of data.
trajectory6 thread 11 assigne

# 3.遇到的一些莫名其妙的错误

### 当我使用for循环时，内存会爆，按道理来说是不会的

In [8]:
# MAIN
START = datetime.datetime.now()
strSTART = START.strftime("%Y-%m-%d %H:%M:%S")
with open (os.path.join(TASK, "%02d_time.log" % ( folderNum )), "a") as l:
    l.write(strSTART + "\n")
#######################################################################
TOTAL = 0
for i in yield_insert(1):
    print(i)
######################################################################    
END = datetime.datetime.now()
strEND = END.strftime("%Y-%m-%d %H:%M:%S")
with open (os.path.join(TASK, "%02d_time.log" % ( folderNum )), "a") as lg:
    lg.write(strEND)
DELTA = (END - START).seconds
print("All threads finished, used %s seconds." % (DELTA))
print("%d pieces of data (%d) has been inserted, %d errors happened." %( TOTAL, col.find().count(), erCol.find().count() ))

Task00 task00
Task00 task01
Task00 task02
Task00 task03trajectory8 thread 2 assigned 52 pieces of data.
trajectory8 thread 0 assigned 2667 pieces of data.

Task00 task04trajectory8 thread 1 assigned 372 pieces of data.

trajectory8 thread 2 assigned 13422 pieces of data.Task00 task05

Task00 task06
trajectory8 thread 3 assigned 2434 pieces of data.
trajectory8 thread 4 assigned 797 pieces of data.
trajectory8 thread 5 assigned 4 pieces of data.
Task00 task07
Task00 task08trajectory8 thread 6 assigned 5 pieces of data.
trajectory8 thread 0 assigned 41652 pieces of data.

Task00 task09trajectory8 thread 7 assigned 2 pieces of data.

trajectory8 thread 1 assigned 42215 pieces of data.
Task00 task10
Task00 task11
trajectory8 thread 7 assigned 2 pieces of data.
trajectory8 thread 3 assigned 35181 pieces of data.trajectory8 thread 8 assigned 223 pieces of data.
Task00 task12
trajectory8 thread 9 assigned 2 pieces of data.
trajectory8 thread 5 assigned 7241 pieces of data.

trajectory8 thread

trajectory8 thread 57 assigned 2 pieces of data.
Task00 task83
trajectory8 thread 80 assigned 13 pieces of data.
trajectory8 thread 17 assigned 84 pieces of data.
trajectory8 thread 27 assigned 5 pieces of data.Task00 task84trajectory8 thread 42 assigned 5455 pieces of data.
trajectory8 thread 59 assigned 3 pieces of data.

trajectory8 thread 81 assigned 7 pieces of data.

Task00 task85trajectory8 thread 83 assigned 6005 pieces of data.

trajectory8 thread 82 assigned 869 pieces of data.
Task00 task86
trajectory8 thread 84 assigned 39 pieces of data.
Task00 task87
trajectory8 thread 58 assigned 2 pieces of data.
trajectory8 thread 86 assigned 7 pieces of data.
Task00 task88trajectory8 thread 60 assigned 12 pieces of data.trajectory8 thread 29 assigned 17 pieces of data.trajectory8 thread 85 assigned 1661 pieces of data.


trajectory8 thread 45 assigned 4 pieces of data.
trajectory8 thread 63 assigned 2 pieces of data.
trajectory8 thread 87 assigned 9 pieces of data.
trajectory8 thread 

trajectory8 thread 62 assigned 2 pieces of data.
trajectory8 thread 93 assigned 2 pieces of data.trajectory8 thread 48 assigned 204 pieces of data.

trajectory8 thread 96 assigned 7 pieces of data.
trajectory8 thread 104 assigned 3 pieces of data.
trajectory8 thread 108 assigned 3433 pieces of data.
trajectory8 thread 116 assigned 2 pieces of data.
trajectory8 thread 52 assigned 95987 pieces of data.
trajectory8 thread 41 assigned 2768 pieces of data.
trajectory8 thread 40 assigned 2 pieces of data.
trajectory8 thread 81 assigned 4 pieces of data.
trajectory8 thread 114 assigned 873 pieces of data.
trajectory8 thread 87 assigned 2 pieces of data.
trajectory8 thread 119 assigned 2 pieces of data.
trajectory8 thread 111 assigned 2 pieces of data.
trajectory8 thread 65 assigned 2 pieces of data.
trajectory8 thread 32 assigned 2 pieces of data.trajectory8 thread 93 assigned 2 pieces of data.

trajectory8 thread 56 assigned 1731 pieces of data.trajectory8 thread 116 assigned 2 pieces of dat

trajectory8 thread 85 assigned 2 pieces of data.
trajectory8 thread 85 assigned 2 pieces of data.
trajectory8 thread 85 assigned 3 pieces of data.
trajectory8 thread 85 assigned 2 pieces of data.
trajectory8 thread 56 assigned 424 pieces of data.
trajectory8 thread 105 assigned 2 pieces of data.
trajectory8 thread 23 assigned 2 pieces of data.
trajectory8 thread 101 assigned 2 pieces of data.
trajectory8 thread 101 assigned 3 pieces of data.
trajectory8 thread 101 assigned 33 pieces of data.
trajectory8 thread 92 assigned 4 pieces of data.
trajectory8 thread 68 assigned 26 pieces of data.
trajectory8 thread 68 assigned 3 pieces of data.
trajectory8 thread 68 assigned 10 pieces of data.
trajectory8 thread 108 assigned 2 pieces of data.
trajectory8 thread 31 assigned 12 pieces of data.
trajectory8 thread 31 assigned 2 pieces of data.
trajectory8 thread 13 assigned 4120 pieces of data.
trajectory8 thread 24 assigned 18 pieces of data.
trajectory8 thread 24 assigned 2 pieces of data.
traje

trajectory8 thread 59 assigned 10128 pieces of data.
trajectory8 thread 4 assigned 2 pieces of data.
trajectory8 thread 4 assigned 1608 pieces of data.
trajectory8 thread 4 assigned 18 pieces of data.
trajectory8 thread 4 assigned 2 pieces of data.
trajectory8 thread 117 assigned 3 pieces of data.
trajectory8 thread 117 assigned 3 pieces of data.
trajectory8 thread 117 assigned 125 pieces of data.
trajectory8 thread 117 assigned 3 pieces of data.
trajectory8 thread 98 assigned 3 pieces of data.
trajectory8 thread 98 assigned 2 pieces of data.
trajectory8 thread 98 assigned 3 pieces of data.
trajectory8 thread 127 assigned 296 pieces of data.
trajectory8 thread 127 assigned 2 pieces of data.
trajectory8 thread 123 assigned 2 pieces of data.
trajectory8 thread 123 assigned 2 pieces of data.
trajectory8 thread 123 assigned 2 pieces of data.
trajectory8 thread 123 assigned 2 pieces of data.
trajectory8 thread 52 assigned 2 pieces of data.
trajectory8 thread 120 assigned 2 pieces of data.
t

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


trajectory8 thread 71 assigned 2 pieces of data.
trajectory8 thread 3 assigned 2 pieces of data.
trajectory8 thread 11 assigned 39372 pieces of data.
trajectory8 thread 11 assigned 2 pieces of data.
trajectory8 thread 11 assigned 2 pieces of data.
trajectory8 thread 11 assigned 3 pieces of data.


### for在数据非常多时很占内存，然后用yeild生成器执行十次main， 不报错，但是发现服务器一条数据也没插入，莫名其妙

In [7]:
def yield_insert(n):
    for i in range(n):
        yield main(i)
    print("all data have inserted") 
    print("end")

### 下面是为断点续插，插入进度可视化的探索，但是不能插入数据，弃用了

In [ ]:
class threadInsert(threading.Thread):
    def __init__(self, queue, fileList, col, erCol, strtask):
        threading.Thread.__init__(self)
        self.queue = queue
        self.fileList = fileList
        self.col = col
        self.erCol = erCol
        self.strtask = strtask

    def run(self):
        total = 0
        with open(self.fileList, "r") as taskList:
            taskList = taskList.read().split("\n")
            
        for txt in taskList:
            if txt == "":
                continue
            else:
                with open(txt, "r") as data:
                    data = data.readlines()
                month = strtask + " "+ os.path.dirname(txt)[-11:]
                print("%s thread %s assigned %s pieces of data." % (month, self.queue, len(data) ))
                total += len(data)

                col_insert = []
                erCol_insert = []
                for c in data:
                    a = insertData(c, self.col, self.erCol, col_insert, erCol_insert)

        self.result = total

    def getResult(self):
        try:
            return self.result
        except:
            return Exception


In [ ]:
START = datetime.datetime.now()
strSTART = START.strftime("%Y-%m-%d %H:%M:%S")

with open (os.path.join(TASK, "%02d_time.log" % ( folderNum )), "a") as tlog:
    tlog.write(strSTART + "\n")
############################################

TOTAL = 0
#for j in range(threadNum):
TaskFolder = os.path.join(TASK, "folder%02d" %(folderNum), "Task00")
threads = []
try:
    for i in range(threadNum):
        queue = "%02d" % ( i )
        taskFile = os.path.join(TaskFolder, "task%s.txt" % ( queue ))
        strtask = "Task%02d "%(j) + taskFile[-10:]
        t = threadInsert(queue=i, fileList=taskFile, col=col, erCol=erCol, strtask=strtask)
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
        result = t.getResult()
        TOTAL += result
        print("finished %d pieces of data." % ( result ))
except:
    CUR = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(os.path.join(TASK, "error.log"), "a") as erLog:
        erLog.write(CUR + "\n")
        erLog.write("%02d" % ( folderNum ) + " - " + str(Exception) + "\n")
#############################################  

END = datetime.datetime.now()
strEND = END.strftime("%Y-%m-%d %H:%M:%S")

with open (os.path.join(TASK, "%02d_time.log" % ( folderNum )), "a") as tlog:
    tlog.write(strEND)
    
DELTA = (END - START).seconds

print("All threads finished, used %s seconds." % (DELTA))
print("%d pieces of data (%d) has been inserted, %d errors happened." %( TOTAL, col.find().count(), erCol.find().count() ))

In [ ]:
START = datetime.datetime.now()
strSTART = START.strftime("%Y-%m-%d %H:%M:%S")
with open (os.path.join(TASK, "%02d_time.log" % ( folderNum )), "w") as tlog:
    tlog.write(strSTART + "\n")

Total = 0
try:
    for j in range(16, threadNum):
        threads = []
        TOTAL = 0
        startque = datetime.datetime.now()
        queues = "%02d" % ( j )
        taskFile = os.path.join(TASK, "folder%02d" %(folderNum), "task%s.txt" % ( queues ))
#        if os.path.exist(taskFile):
        strtask = taskFile[-10:]
        for i in range(threadNum):
            t = threadInsert(queue=i, fileList=taskFile, col=col, erCol=erCol, strtask=strtask)
            t.start()
            threads.append(t)
        os.remove(taskFile)
        costque = (datetime.datetime.now() - startque).seconds
        for t in threads:
            t.join()
            result = t.getResult()
            TOTAL += result
        Total += TOTAL
        rstr = strtask[:-4] + "finished %d pieces of data." % ( TOTAL )
        with open (os.path.join(TASK, "%02d_time.log" % ( folderNum )), "a") as l:
            l.write(rstr + "cost %s seconds "%(costque) + "\n")
        # print("%d pieces of data finished, %d errors." % ( col.find().count(), erCol.find().count() ))

except:
    CUR = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(os.path.join(TASK, "error.log"), "a") as erLog:
        erLog.write(CUR + "\n")
        erLog.write("%02d" % ( folderNum ) + " - " + str(Exception) + "\n")

END = datetime.datetime.now()
strEND = END.strftime("%Y-%m-%d %H:%M:%S")
with open (os.path.join(TASK, "%02d_time.log" % ( folderNum )), "a") as tlog:
    tlog.write(strEND)
    
DELTA = (END - START).seconds
print("All threads finished, used %s seconds." % (DELTA))
print("%d pieces of data (%d) has been inserted, %d errors happened." %( Total, col.find().count(), erCol.find().count() ))


In [ ]:
def initTask(dataDir, taskListDir):
    taskdir = os.path.join(taskListDir, "folder%02d" % (folderNum))   #/root/ais/Projects/month%d/folder%d
    if os.path.exists(taskdir):
        print("spliting files finashed.\n")
        return

    filePath = []       #保存所有txt文件的绝对路径

    for root, dirs, files in os.walk(dataDir):      #扫描所有数据文件名
        for file in files:  
            if file.endswith('txt'):
                filePath.append(os.path.join(root, file))   

    print("All txt: %d" % ( len(filePath) ))

    for i in range(threadNum):          #循环执行所有线程
        queue = "%02d" % ( i )
        createVar["fileQueue" + queue] = []    #生成threadNum个list 保存filepath
        
    for i in range(len(filePath)):      #将所有txt文件分为 Threadnum 份
        queue = "%02d" % (i % threadNum)
        createVar["fileQueue" + queue].append(filePath[i])

    for i in range(threadNum):          #
        queue = "%02d" % ( i )
        if not os.path.isdir(taskdir):           #/root/ais/Projects/month%d/folder%d/Task%s/ 不存在则创建
            os.mkdir(taskdir)
        tasknum = os.path.join(taskdir,"Task%s"%(queue))
        if not os.path.isdir(tasknum):
            os.mkdir(tasknum)
            #文件夹task00:
            
        for j in range(threadNum):
            que = "%02d" % ( j )
#            if not os.path.isdir(taskdir,"Task%s"%(queue)):     #/root/ais/Projects/month%d/folder%d/Task%s/ 不存在则创建
 #               os.mkdir(taskdir,"Task%s"%(queue))
                
            fn = os.path.join(taskdir ,"Task%s"%(queue),"task%s.txt" % (que))    #folder%d/task%s.txt
            if os.path.isfile(fn):                              #删除目录下已存在的task%s.txt文件
                os.remove(fn)
                
            for m in range(threadNum):          #
                Queue = "%02d" % ( m )
                Var["fileQueue" + Queue] = []
                #128*Var[] = createVar[]
                
            for k in range(len(createVar["fileQueue" + queue])):      #createVar[00] 
                qu = "%02d" % (k % threadNum)
                Var["fileQueue" + qu].append(createVar["fileQueue" + queue][k])
                
            with open(fn, "a") as task:                   #将createVar["fileQueue" + queue]写入task%s.txt
                for file in Var["fileQueue" + que]:      
                    task.write(file + "\n")

            print("Task%s task%s: %d" % (queue, que, len(Var["fileQueue" + que])))
            Var["fileQueue" + que] = []
            fn = []
        
    # release memory        
    createVar["fileQueue" + queue] = []

    print("spliting files finished.\n")

    # release memory
    filePath = []

